Our problematic is that density is really respected in our network as long as it evolves.  
We must, therefore, find rules such that it can work properly, having nice and realistic 2D representation.

In [1]:
from gpn6 import GrowingPlanarNetwork, gpn_action
from helper import *
import networkx as nx
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from heapq import heappush, heappop
import gc

In [2]:
def p_dupl():
    return np.random.random() > 0.46

In [3]:
def std4(distr):
    distr = np.asarray(distr)
    mean = np.mean(distr)
    dev = (distr - mean)**4
    return np.mean(dev)

In [4]:
class EnergyPoint:
    def __init__(self, node, axis, val):
        self.node = node
        self.axis = axis
        self.val = val
        
    def to_shorten(self):
        return self.val > 1.
    
    def get_norm_val(self):
        if self.to_shorten():
            return 1 / self.val
        return self.val
        
    def __gt__(self, other):
        return self.get_norm_val() > other.get_norm_val()
    
    def __lt__(self, other):
        return self.get_norm_val() < other.get_norm_val()
    
    def __eq__(self, other):
        return self.get_norm_val() == other.get_norm_val()
    
class RelaxAction:
    """
    Handles :
    
    """
    def __init__(self, name, args, pattern):
        self.name = name
        self.args = args
        self.pattern = pattern
        self.score = None
        
    def __gt__(self, other):
        return self.score > other.score
    
    def __lt__(self, other):
        return self.score < other.score
    
    def __eq__(self, other):
        return self.score == other.score
        
    def more(self, node):
        nodes = set()
        for sign, edge in self.pattern:
            if sign == "+" and node in edge:
                nodes |= set(edge)
        nodes -= {node}
        return nodes
    
    def less(self, node):
        nodes = set()
        for sign, edge in self.pattern:
            if sign == "-" and node in edge:
                nodes |= set(edge)
        nodes -= {node}
        return nodes
    
    def get_nodes(self):
        nodes = set()
        for _, edge in self.pattern:
            nodes |= set(edge)
            
        return nodes
    
    def __repr__(self):
        return f"RelaxAction : {self.name} with {self.args}, pattern is {self.pattern} : {self.score}"

In [5]:
class RGPN(GrowingPlanarNetwork):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_action = None
        self.trials = list()
        
    def init_tissue(self, size=3):
        super().init_tissue(size=size)
        total_size = size**2 - 1
        # set north
        for i in range(size):
            self.G.nodes[i]["side"] = self.G.nodes[i].get("side", set()) | {"N"}
        
        # set south
        for i in range(size):
            self.G.nodes[total_size - i]["side"] = self.G.nodes[total_size - i].get("side", set()) | {"S"}
        
        # set west
        for i in range(size):
            self.G.nodes[i * size]["side"] = self.G.nodes[i * size].get("side", set()) | {"W"}
        
        # set east
        for i in range(size):
            self.G.nodes[total_size - i * size]["side"] = \
                self.G.nodes[total_size - i * size].get("side", set()) | {"E"}
            
    def diffuse_border(self, node):
        # if the border is a corner, we can just return
        # ideally, we shall never have a length of 3 ...
        
        if self.is_border_node(node):
            if len(self.G.nodes[node].get("side", set())) >= 2:
                return
            ngb1, ngb2 = self.get_border_neighbours(node)
            side1 = self.G.nodes[ngb1].get("side", set())
            side2 = self.G.nodes[ngb2].get("side", set())
            side = self.G.nodes[node].get("side", set())
            assert len(side1 & side2) == 1, f"side is not of the right size, " \
                f"should be one : {side1}, {side2}, {ngb1}, {ngb2}" \
                f" with node {node}, {side}"
            self.G.nodes[node]["side"] = side1 & side2
            
        else:
            if "side" in self.G.nodes[node]:
                del self.G.nodes[node]["side"]
    
    def update_corner(self, node):
        if self.is_border_node(node):
            ngb1, ngb2 = self.get_border_neighbours(node)
            side1 = self.G.nodes[ngb1].get("side", set())
            side2 = self.G.nodes[ngb2].get("side", set())
            side_ref = self.G.nodes[node].get("side", set())
            
            if len(side1 & side2) == 0:
                self.G.nodes[ngb1]["side"] = side1 | side2
            
    # TODO override duplicate to set the border (NSEW)
    def duplicate(self, node):
        new_node = super().duplicate(node)
        self.diffuse_border(new_node)
            
    
    # TODO override remove to check for corners
    def destroy(self, node):
        self.update_corner(node)
        super().destroy(node)
    
    def edge_swapper(self):
        pass
    
    def update_all_dist(self):
        for direction in "NESW":
            self.update_dist(direction=direction)
        
    def mean_NS(self):
        return np.mean([self.G.nodes[n]["dist_N"] + self.G.nodes[n]["dist_S"] for n in self.G.nodes])
    
    def mean_EW(self):
        return np.mean([self.G.nodes[n]["dist_E"] + self.G.nodes[n]["dist_W"] for n in self.G.nodes])
    
    def update_dist(self, direction="N"):
        """
        Update for each node its distance to each sides
        Direction is one of NSEW
        """
        key = "dist_" + direction
        queue = []
        nodes = set(self.G.nodes)
        # NORTH
        for i in self.G.nodes:
            n = self.G.nodes[i]
            if direction in n.get("side", set()):
                value = 0
            else:
                value = np.inf
                
            n[key] = value
            heappush(queue, (value, i))
            
        while queue and nodes:
            _, inode = heappop(queue)
            if inode not in nodes:
                continue
                
            nodes.remove(inode)
            node = self.G.nodes[inode]
            value = node[key] + 1
            for ngb in node["ngb"]:
                if value < self.G.nodes[ngb][key]:
                    self.G.nodes[ngb][key] = value
                    heappush(queue, (value, ngb))

    
    def show_dist(self, k=5, iterations=1000, figsize=(10, 10), pos=None, hl_nodes=[],
                 print_dist=True):
        # green for NS, red for EW
        plt.figure(figsize=figsize)
        if not pos:
            pos = nx.spring_layout(self.G, k=k, iterations=iterations)
        green = lambda x: format(max(0, 255 - (x["dist_N"] + x["dist_S"]) * 20), '02x')
        red = lambda x: format(max(0, 255 - (x["dist_E"] + x["dist_W"]) * 20), '02x')
        corner = lambda x: len(x.get("side", set())) >= 2
        
        def col_border(node):
            side = tuple(node.get("side", set()))
            return {
                ("N",): "#888822",  # yellow
                ("S",): "#228888",  # cyan
                ("E",): "#882288",  # purple
                ("W",): "#666666",  # grey
            }.get(side, "")
        
        def col_other(n):
            if n in hl_nodes:
                return "#FF4444"  # red
            return "#" + red(self.G.nodes[n]) + green(self.G.nodes[n]) + "ff"
            
        colors = ["#000000" if corner(self.G.nodes[n]) else 
                  col_border(self.G.nodes[n])
                  or col_other(n) for n in self.G.nodes]
        
        if print_dist:
            xi = 0.05
            delta = dict(N=np.array([-xi, 0]), S=np.array([xi, 0]),
                        E=np.array([0, -xi]), W=np.array([0, xi]))
            for d in "NESW":
                for n in self.G.nodes:
                    x, y = pos[n] + delta[d]
                    plt.text(x, y, self._dist(n, d))
                    
        
        nx.draw_networkx(self.G, pos, node_color=colors)
                
    def _get_border_candidates(self, node):
        assert self.is_border_node(node)
        return [n for ngb in gpn.get_border_neighbours(node)
                for n in gpn.get_border_neighbours(ngb)
                if n != node]
    
    def _get_quick_candidates(self, node, dual_node):
        assert dual_node != -1, "Operation not allowed for dual -1"
        return [ngb for ngb in 
                self.get_intermediate_neighbours(dual_node, net="dual")
                if ngb != node]
    
    def _get_common_border_ngb(self, n_a, n_b):
        s_a = set(gpn.get_border_neighbours(n_a))
        s_b = set(gpn.get_border_neighbours(n_b))
        return list(s_a & s_b)[0]
    
    def get_func_action(self, name, pattern=False):
        if name == "build":
            if pattern:
                return self.build_new_edge_pattern
            else:
                return self.build_new_edge
        
        elif name == "replace":
            if pattern:
                return self.replace_edge_pattern
            else:
                return self.replace_edge
        
        elif name == "setup":
            if pattern:
                return self.setup_edge_pattern
            else:
                return self.setup_edge
    
    def build_new_edge_pattern(self, n, candidate, dual):
        pattern = [("+", (n, candidate))]
        return pattern
        
    @gpn_action
    def build_new_edge(self, n, candidate, dual):
        if (n, candidate) in self.G.edges:
            return
        
        if dual == -1:
            ref_node = self._get_common_border_ngb(n, candidate)
            if self.is_corner(ref_node):
                self.update_corner(ref_node)
            self._make_border_edge(ref_node, (n, candidate))
            self.diffuse_border(ref_node)
        else:
            self._make_edge(dual, (n, candidate))
            
        self.stabilize_around([n, candidate])
            
        return True
    
    def replace_edge_pattern(self, edge1, edge2):
        pattern = [("+", edge1), ("-", edge2)]
        return pattern
                
    @gpn_action
    def replace_edge(self, edge1, edge2):
        dual_node = self._remove_edge(edge1)
        # print("Dual node", dual_node)
        self._make_edge(dual_node, edge2)
        self.stabilize_around(list(set(edge1) | set(edge2)))
        
    def setup_edge_pattern(self, edge, dual_node, candidate):
        pattern = list()
        # setup the edge and delete the one chosen
        node1, node2 = edge
        if (node1, candidate) not in self.G.edges:
            pattern.append(("+", (node1, candidate)))
            
        if (node2, candidate) not in self.G.edges:
            pattern.append(("+", (node2, candidate)))
        
        self._remove_edge(edge)
        pattern.append(("-", edge))
        return pattern
    
    @gpn_action
    def setup_edge(self, edge, dual_node, candidate):
        # setup the edge and delete the one chosen
        node1, node2 = edge
        if (node1, candidate) not in self.G.edges:
            other_dual = self._make_edge(dual_node, (node1, candidate))
            # update dual_node
            ingb = set(self.get_intermediate_neighbours(dual_node, net="dual"))
            if not ingb.issuperset({node2, candidate}):
                dual_node, other_dual = other_dual, dual_node
            
        if (node2, candidate) not in self.G.edges:
            self._make_edge(dual_node, (node2, candidate))
        
        self._remove_edge(edge)
        self.diffuse_border(candidate)
        self.stabilize_around(list(set(edge) | {candidate}))
        
    def relax(self):
        # prepare
        self.last_action = None
        self.update_all_dist()
        mean_NS = self.mean_NS()
        mean_EW = self.mean_EW()
        
        nodes = list(self.G.nodes)
        random.shuffle(nodes)
        
        # list all pain points
        candidates = list()
        for n in nodes:
            node = self.G.nodes[n]
            val_NS = (node["dist_N"] + node["dist_S"]) / mean_NS
            val_EW = (node["dist_E"] + node["dist_W"]) / mean_EW
            if val_NS >= 1.1: candidates.append(EnergyPoint(n, "NS", val_NS))
            if val_EW >= 1.1: candidates.append(EnergyPoint(n, "EW", val_EW))
            if val_NS <= 0.9: candidates.append(EnergyPoint(n, "NS", val_NS))
            if val_EW <= 0.9: candidates.append(EnergyPoint(n, "EW", val_EW))
                
        candidates.sort(reverse=True)
        
        # list all possible actions
        action_candidates = list()
        for pt in candidates[:10]:
            for action in self.enumerate_actions(pt):
                self.estimate_action(action, mean_NS, mean_EW)
                action_candidates.append(action)
                
        # higher score is better !!
                
        action_candidates.sort(reverse=True)
        
        for action in action_candidates[:20]:
            self.try_action(action)
            
        self.keep_best_trial()
        
    def enumerate_actions(self, pt):
        if pt.to_shorten():
            for x in self._shorten(pt):
                yield x
        else:
            for x in self._maxen(pt):
                yield x
                
    def _quick_shorten(self, pt):
        ingbs = self.get_intermediate_neighbours(pt.node)
        
        # gather all candidates without any filter
        candidates = []
        for ingb in ingbs:
            if ingb == -1:
                ls_c = self._get_border_candidates(pt.node)
            else:
                ls_c = self._get_quick_candidates(pt.node, ingb)
            for c in ls_c:
                args = (pt.node, c, ingb)
                action = RelaxAction(name="build", args=args, 
                                     pattern=self.build_new_edge_pattern(*args))
                yield action
                
    def _shorten(self, pt):
        # maybe it could be simplified later
        for x in self._quick_shorten(pt):
            yield x
            
        ingbs = set(self.get_intermediate_neighbours(pt.node)) - {-1}
        
        for ingb in ingbs:
            dngbs = self.ngb(ingb, net="dual")
            
            for edge_dngb in dngbs:
                node_dngb = self.get_other_node(edge_dngb, ingb)
                if node_dngb in (set(ingbs) | {-1}):
                    continue
                    
                # if #ngb == 2
                p_edge = self.dual(edge_dngb)
                if len(self.ngb(p_edge[0])) == 2 or len(self.ngb(p_edge[1])) == 2:
                    continue
                    
                pngbs = self.get_intermediate_neighbours(node_dngb, net="dual")
                
                for pngb in pngbs:
                    # no edge
                    if (pt.node, pngb) in self.G.edges:
                        continue
                        
                    # HERE creating action
                    args = (self.dual(edge_dngb), (pt.node, pngb))
                    action = RelaxAction(name="replace", args=args, 
                                         pattern=self.replace_edge_pattern(*args))
                    
                    yield action
    
    def _maxen(self, pt):
        if len(self.ngb(pt.node)) == 2:
            return
        
        for ngb in self.ngb(pt.node):
            if len(self.ngb(ngb)) == 2:
                continue
                
                dual_nodes = set(self.dual((pt.node, ngb))[:2]) - {-1}
                for dual_node in dual_nodes:
                    pngbs = set(self.get_intermediate_neighbours(dual_node, net="dual")) - {pt.node, ngb}
                    for pngb in pngbs:
                        args = ((pt.node, ngb), dual_node, pngb)
                        action = RelaxAction(name="setup", args=args,
                                            pattern=self.setup_edge_pattern(*args))
                        
                        yield action
                        
    def estimate_action(self, action, mean_NS, mean_EW):
        old_score_NS = []
        new_score_NS = []
        old_score_EW = []
        new_score_EW = []
        for node in action.get_nodes():
            ngbs = set(self.ngb(node)) | action.more(node) - action.less(node)
            old_score_NS.append(self._dist(node, "NS"))
            old_score_EW.append(self._dist(node, "EW"))
            dist = dict(N=1e9, E=1e9, S=1e9, W=1e9)
            for ngb in ngbs:
                for direction in "NESW":
                    dist[direction] = min(dist[direction], self._dist(ngb, direction) + 1)
            new_score_NS.append(dist["N"] + dist["S"])
            new_score_EW.append(dist["W"] + dist["E"])
                
        
        score = np.sum((np.array(old_score_NS) - mean_NS)**2)
        score += np.sum((np.array(old_score_EW) - mean_EW)**2)
        score -= np.sum((np.array(new_score_NS) - mean_NS)**2)
        score -= np.sum((np.array(new_score_EW) - mean_EW)**2)
        action.score = score
    
    def try_action(self, action):
        gpn = self.copy()
        gpn.run_action(action)
        self.trials.append(gpn)
    
    def run_action(self, action):
        func = self.get_func_action(action.name)
        self.last_action = action
        func(*action.args)
        
    def keep_best_trial(self):
        # a bit ugly but should work
        print([(x.get_main_metric(update=True), len(x.G.edges)) for x in self.trials])
        print(self.get_main_metric(update=True), len(self.G.edges))
        best = min(self.trials + [self], key=lambda x: x.get_main_metric(update=True))
        self.trials.clear()
        print(f"Action ran : {best.last_action}")
        self.__dict__ = best.__dict__  # maybe use __dict__
    
    def get_all_dists(self):
        l_NS, l_EW, l_both = list(), list(), list()
        nodes = list(self.G.nodes)
        candidates = list()
        for n in nodes:
            node = self.G.nodes[n]
            val_NS = (node["dist_N"] + node["dist_S"])
            val_EW = (node["dist_E"] + node["dist_W"])
            l_NS.append(val_NS)
            l_EW.append(val_EW)
            l_both.append(val_NS + val_EW)
            
        return l_NS, l_EW, l_both
    
    def get_main_metric(self, update=False):
        if update:
            self.update_all_dist()
        lA, lB, lC = self.get_all_dists()
        # xA, xB, xC = np.std(lA) / np.mean(lA), np.std(lB) / np.mean(lB), np.std(lC) / np.mean(lC)
        xA, xB, xC = std4(lA), std4(lB), std4(lC)
        return xA + xB + xC
    
    def print_dist_metrics(self):
        lA, lB, lC = self.get_all_dists()
        xA, xB, xC = np.std(lA) / np.mean(lA), np.std(lB) / np.mean(lB), np.std(lC) / np.mean(lC)
        print(f"Score are for NS : {xA}, EW : {xB}, both : {xC}")
        return xA + xB + xC
    
    # TODO, find a better way
    def _dist(self, n, axis):
        node = self.G.nodes[n]
        return sum([node["dist_" + str(x)] for x in axis])
    
    def is_useless_edge(self, edge):
        return (self.use_other_node(edge[0], edge[1]) or
                self.use_other_node(edge[1], edge[0]))
    
    # NOT WORKING
    def use_other_node(self, node1, node2):
        # test for each direction that you can find
        # the proper value of dist
        nodes = set(self.ngb(node1)) - {node2}
        for d in ["dist_E", "dist_W", "dist_N", "dist_S"]:
            for n in nodes:
                if self.G.nodes[node2][d] == self.G.nodes[n][d] + 1:
                    continue
            return False
        return True
    
    def remove_useless_edge(self):
        edges = list(self.G.edges)
        random.shuffle(edges)
        
        for e in edges:
            if self.is_useless_edge(e):
                self._remove_edge(e)
                return True

In [6]:
def node_pos(node):
    x = node["dist_N"] / (node["dist_N"] + node["dist_S"])
    y = node["dist_E"] / (node["dist_E"] + node["dist_W"])
    x, y = (x - 0.5) * 1.5, (y - 0.5) * 1.5
    return np.array([x, y])

def embedded_viz(gpn, finetune=True):
    gpn.update_all_dist()
    # pos is a dict of np array with coordinates
    pos = {n: node_pos(gpn.G.nodes[n]) for n in gpn.G.nodes}
    if finetune:
        pos = pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=1)
    return pos

def quick_export(gpn, pos, name, useful=None):
    global ignore_export
    if ignore_export:
        return
    gpn.update_all_dist()
    gpn.show_dist(pos=pos, figsize=(12, 12))
    color = {True: "green", False: "red"}.get(useful, "grey")
    rect = plt.Rectangle((-0.825, 0.750), 0.075, 0.075, fc=color)
    plt.gca().add_patch(rect)
    plt.savefig(name)
    plt.close()
    
def print_run(i):
    print()
    print("-" * 50)
    print()
    print(f"Run {i}")
    
def get_pathological_nodes_1(gpn):
    # delta dist with ngb >= 2 in one direction
    pathological_nodes = list()
    for n in gpn.G.nodes:
        ref = gpn._dist(n, axis="NS"), gpn._dist(n, axis="EW")
        # calc center of mass of ngbs
        for ngb in gpn.ngb(n):
            d = gpn._dist(n, axis="NS"), gpn._dist(n, axis="EW")
            if (d[0] >= ref[0] + 2) or (d[0] >= ref[0] + 2):
                pathological_nodes.append(n)
                break
            
    return pathological_nodes

def get_pathological_nodes_2(gpn):
    # center of gravity too much "out"
    pathological_nodes = list()
    for n in gpn.G.nodes:
        if gpn.is_border_node(n):
            continue
        c = np.array([0, 0])
        # calc center of mass of ngbs
        for i, ngb in enumerate(gpn.ngb(n)):
            node = gpn.G.nodes[ngb]
            c = c + node_pos(node)
        c = c[0] / (i + 1), c[1] / (i + 1)
        
        # compare
        node = gpn.G.nodes[n]
        ref = node_pos(node)
        dist = np.sqrt((c[0] - ref[0])**2 + (c[1] - ref[1])**2)

        if dist > 0.1:
            pathological_nodes.append(n)
            
    return pathological_nodes

def get_marginal_nodes(gpn, thr=1.1):
    test = (lambda x: x > thr) if thr > 1 else (lambda x: x < thr)
    mean_NS, mean_EW = gpn.mean_NS(), gpn.mean_EW()
    
    candidates = list()
    for n in gpn.G.nodes:
        node = gpn.G.nodes[n]
        val_NS = (node["dist_N"] + node["dist_S"]) / mean_NS
        val_EW = (node["dist_E"] + node["dist_W"]) / mean_EW
        if test(val_NS):
            candidates.append(n)

        if test(val_EW):
            candidates.append(n)
            
    return candidates

In [10]:
seed = 3
random.seed(seed)
np.random.seed(seed)
gpn = RGPN()
gpn.debug = False
gpn.init_tissue(8)
gpn.update_all_dist()
ignore_export = False

pos = nx.spring_layout(gpn.G, k=0.1, iterations=50)
root = "output/evonet11/"
ratio = 0
for i in range(300):
    print_run(i)
    if i < 250:
        if p_dupl():
            ratio += 1
            gpn.duplicate_random()
        else:
            ratio -= 1
            gpn.destroy_random()
        
    pos = embedded_viz(gpn, finetune=False)
    quick_export(gpn, pos, root + str(i).zfill(4) + "_a.png")
    
    print("Action")
    gpn.update_all_dist()
    dist_score = gpn.print_dist_metrics()
    
    gpn.relax()
    
    gpn.update_all_dist()
    dist_score = gpn.print_dist_metrics()
    pos = embedded_viz(gpn, finetune=False)
    quick_export(gpn, pos, root + str(i).zfill(4) + "_b.png")
    
    """

    if gpn.shorten_dist():
        pos = embedded_viz(gpn)
        print("Shorten")
        gpn.update_all_dist()
        sc = gpn.print_dist_metrics()
        quick_export(gpn, pos, root + str(i).zfill(4) + "_b.png", useful=sc < dist_score)
        dist_score = sc

    if gpn.maxen_dist():
        pos = embedded_viz(gpn)
        print("Maxen")
        gpn.update_all_dist()
        sc = gpn.print_dist_metrics()
        quick_export(gpn, pos, root + str(i).zfill(4) + "_c.png", useful=sc < dist_score)
        dist_score = sc
        
    if gpn.remove_useless_edge():
        pos = embedded_viz(gpn)
        quick_export(gpn, pos, root + str(i).zfill(4) + "_d.png")
        print("AAAAAAAAAAAAAA - Remuseless")
        gpn.update_all_dist()
        gpn.print_dist_metrics()
        
    """
    if i % 10 == 0 or True:
        gc.collect()
        
    if i % 100 == 0:
        print()
        print("Iteration", i, "ratio", ratio, "density", gpn.density())


--------------------------------------------------

Run 0
Action
Score are for NS : 0.0, EW : 0.0, both : 0.0
[]
0.0 112
Action ran : None
Score are for NS : 0.0, EW : 0.0, both : 0.0

Iteration 0 ratio -1 density 0.08035714285714286

--------------------------------------------------

Run 1
Action
Score are for NS : 0.0, EW : 0.045951795949294105, both : 0.022789102056153986
[]
0.14012472374748358 110
Action ran : None
Score are for NS : 0.0, EW : 0.045951795949294105, both : 0.022789102056153986

--------------------------------------------------

Run 2
Action
Score are for NS : 0.0, EW : 0.06211019534324773, both : 0.030537512710430148
[]
0.16604639617051775 110
Action ran : None
Score are for NS : 0.0, EW : 0.06211019534324773, both : 0.030537512710430148

--------------------------------------------------

Run 3
Action
Score are for NS : 0.04930096990483421, EW : 0.07221051701797289, both : 0.04630470590245594
[(0.6748017911882893, 112), (0.6748017911882893, 112), (0.767514355385


--------------------------------------------------

Run 12
Action
Score are for NS : 0.09892506036723674, EW : 0.11273488394521763, both : 0.07349262476831549
Quasi lonely node 52 with #ngbs = 2
[(3.741280639239824, 121), (5.930955456564643, 120), (4.6804160817766265, 121), (6.952780496314311, 123), (4.588767787846361, 122), (4.588767787846361, 122), (4.854305295366377, 121), (4.6804160817766265, 122), (5.5115154541422395, 121), (5.7045820147717, 121), (4.6804160817766265, 121), (4.844970288442916, 121), (4.81421340407051, 121), (5.5115154541422395, 121), (5.379162050867769, 121), (4.768815462693015, 121), (4.768815462693015, 121), (4.683196892317579, 121), (4.6804160817766265, 121), (5.010143426369176, 121)]
4.6804160817766265 121
Action ran : RelaxAction : build with (69, 44, 39), pattern is [('+', (69, 44))] : 3.079365079365079
Score are for NS : 0.09391307671782252, EW : 0.11295802570395012, both : 0.06985005533188726

--------------------------------------------------

Run 13
Act


--------------------------------------------------

Run 31
Action
Score are for NS : 0.08786762268960924, EW : 0.1619683095009284, both : 0.09241263586955233
Quasi lonely node 75 with #ngbs = 2
Quasi lonely node 75 with #ngbs = 2
Quasi lonely node 75 with #ngbs = 2
Removing crossing border edge 6 78 (3, 79, 0)
Removing crossing border edge 6 78 (3, 79, 0)
Removing crossing border edge 3 78 (64, 79, 0)
Calling outside_with_anchor
debug_anchor {'next_': 2, 'prev': 70, 'current': 3, 'ngb2': 3, 'ngb1': 78, 'node': 70, 'self': <__main__.RGPN object at 0x7f6b91f38588>}
Quasi lonely node 70 with #ngbs = 2
Quasi lonely node 75 with #ngbs = 2
[(10.937148830400178, 118), (11.752844954256592, 118), (11.17079105402043, 118), (10.904133338603021, 118), (11.213984305716926, 118), (10.937148830400178, 119), (9.856863704236288, 119), (12.761201017627108, 117), (10.044459871513482, 118), (10.044459871513482, 118), (10.888146696176914, 118), (10.888146696176914, 118), (10.8828039643928, 119), (10.90413


--------------------------------------------------

Run 49
Action
Score are for NS : 0.14297330820226312, EW : 0.22133727843035036, both : 0.1502058145994372
[]
35.370701442170265 125
Action ran : None
Score are for NS : 0.14297330820226312, EW : 0.22133727843035036, both : 0.1502058145994372

--------------------------------------------------

Run 50
Action
Score are for NS : 0.14649987945167034, EW : 0.23096223594052653, both : 0.15861899459910747
[]
49.59064758729799 126
Action ran : None
Score are for NS : 0.14649987945167034, EW : 0.23096223594052653, both : 0.15861899459910747

--------------------------------------------------

Run 51
Action
Score are for NS : 0.19906541544688178, EW : 0.20448731984093488, both : 0.1756156592202274
Removing crossing border edge 70 16 (94, 96, 0)
Removing crossing border edge 16 86 (85, 96, 0)
Removing crossing border edge 2 70 (8, 2, 0)
[(66.10565933565557, 125), (64.33437650990243, 125), (74.2581484137645, 125), (64.29498560612386, 126), (78.0

[(11.733027762987632, 136), (12.943680320054764, 136), (11.16155615880817, 136), (12.943680320054764, 135), (15.016230777609563, 134), (12.943680320054764, 135), (9.442982106108667, 135), (13.067409045903977, 136), (12.300433036908391, 136), (12.300433036908395, 136), (11.633848162719914, 136), (11.633848162719914, 136), (12.943680320054764, 135), (12.943680320054764, 135), (13.450737833004906, 136), (12.943680320054764, 135), (12.943680320054764, 135), (30.275447420075977, 134), (12.943680320054764, 135), (12.943680320054764, 135)]
12.943680320054764 135
Action ran : RelaxAction : replace with ((37, 84), (52, 62)), pattern is [('+', (37, 84)), ('-', (52, 62))] : 2.840579710144928
Score are for NS : 0.1123161829935844, EW : 0.1635929440198408, both : 0.09776312272128923

--------------------------------------------------

Run 61
Action
Score are for NS : 0.11303985652347942, EW : 0.1554735860211007, both : 0.09564285976986227
[(11.806292448156753, 135), (8.06092676168269, 135), (9.5374

...

Action ran : RelaxAction : build with (103, 46, 283), pattern is [('+', (103, 46))] : 1.9777777777777779
Score are for NS : 0.11091021036245084, EW : 0.07735430709561472, both : 0.07137921084458772

--------------------------------------------------

Run 220
Action
Score are for NS : 0.10996182385768032, EW : 0.07708715650623477, both : 0.07020843844911144
Calling outside_with_anchor
debug_anchor {'next_': 58, 'prev': 185, 'current': 137, 'ngb2': 137, 'ngb1': 152, 'node': 185, 'self': <__main__.RGPN object at 0x7f6b45ff63c8>}
Quasi lonely node 185 with #ngbs = 2
Removing crossing border edge 137 152 (337, 342, 0)
Removing crossing border edge 185 58 (298, 342, 0)
Removing crossing border edge 128 185 (298, 342, 0)
Calling outside_with_anchor
debug_anchor {'next_': 51, 'prev': 152, 'current': 172, 'ngb2': 172, 'ngb1': 185, 'node': 152, 'self': <__main__.RGPN object at 0x7f6b45ffcba8>}
Quasi lonely node 152 with #ngbs = 2
Removing crossing border edge 185 172 (337, 342, 0)
Removing cross


--------------------------------------------------

Run 228
Action
Score are for NS : 0.11149830688480636, EW : 0.07727091965505185, both : 0.0687156507597844
[(5.245052461641212, 185), (5.675787901651159, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.176478102554082, 185), (5.582773864063881, 186), (5.098113697797073, 185), (5.11652070790095, 187), (5.766997519692356, 185), (5.17743690732832, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.522947945971126, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.766997519692356, 185), (5.766997519692356, 185)]
5.766997519692356 185
Action ran : RelaxAction : replace with ((137, 105), (185, 105)), pattern is [('+', (137, 105)), ('-', (185, 105))] : 2.0989010989010977
Score are for NS : 0.1160458831211787, EW : 0.07566042751698215, both : 0.06500701573081015

--------------------------------------------------

Run 229
Action
Sc

[(21.99515308641975, 182), (23.861213991769546, 183), (26.42838811156836, 182), (24.756377137631457, 183), (20.260581527206213, 183), (21.96206849565615, 183), (22.132684407864655, 182), (23.409354915409235, 183), (22.10934421582076, 182), (21.942991403749424, 182), (21.942991403749424, 182), (21.938042158207587, 183), (21.938042158207583, 183), (25.49721252857796, 183), (21.963046273433925, 183), (21.942991403749424, 182), (30.67504444444444, 184), (22.89652592592592, 182), (26.201998262459988, 182), (20.887712757201644, 182)]
21.942991403749424 182
Action ran : RelaxAction : build with (194, 198, 284), pattern is [('+', (194, 198))] : 2.844444444444446
Score are for NS : 0.13410309368872328, EW : 0.10839989086407258, both : 0.09609364055652697

--------------------------------------------------

Run 248
Action
Score are for NS : 0.12612707925622116, EW : 0.12176459305406868, both : 0.10331676495459849
Calling outside_with_anchor
debug_anchor {'next_': 131, 'prev': 190, 'current': 128


--------------------------------------------------

Run 266
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 267
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 268
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 269
Action
Score are for NS : 0.12263779848516972


--------------------------------------------------

Run 294
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 295
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 296
Action
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364
[]
12.403823167816405 183
Action ran : None
Score are for NS : 0.12263779848516972, EW : 0.11432562466465616, both : 0.08883308497639364

--------------------------------------------------

Run 297
Action
Score are for NS : 0.12263779848516972

In [ ]:
gc.collect()

In [8]:
raise

RuntimeError: No active exception to reraise

In [ ]:
doer = gpn.copy()
(21, 13) in doer.G.edges
# doer.show_all()

In [ ]:
doer = gpn.copy()
doer.check_all()
doer.build_new_edge(*args)

In [ ]:
args = (21, 13, 12)

## Rollback

In [ ]:
seed = 2
random.seed(seed)
np.random.seed(seed)
gpn = RGPN()
gpn.debug = True
gpn.init_tissue(8)
gpn.update_all_dist()
ignore_export = True

pos = nx.spring_layout(gpn.G, k=0.1, iterations=50)
root = "output/evonet8/"
ratio = 0
for i in range(150):
    print_run(i)
    if i < 100:
        if p_dupl():
            ratio += 1
            gpn.duplicate_random()
        else:
            ratio -= 1
            gpn.destroy_random()
        
    pos = embedded_viz(gpn)
    quick_export(gpn, pos, root + str(i).zfill(4) + "_a.png")
    
    print("Action")
    gpn.update_all_dist()
    dist_score = gpn.print_dist_metrics()
    back = gpn.copy()

    if gpn.shorten_dist():
        pos = embedded_viz(gpn)
        print("Shorten")
        gpn.update_all_dist()
        sc = gpn.print_dist_metrics()
        if not sc < dist_score:
            print("Rollback")
            gpn = back
        else:
            quick_export(gpn, pos, root + str(i).zfill(4) + "_b.png", useful=sc < dist_score)
            dist_score = sc
            
    back = gpn.copy()

    if gpn.maxen_dist():
        pos = embedded_viz(gpn)
        print("Maxen")
        gpn.update_all_dist()
        sc = gpn.print_dist_metrics()
        if not sc < dist_score:
            print("Rollback")
            gpn = back
        else:
            quick_export(gpn, pos, root + str(i).zfill(4) + "_c.png", useful=sc < dist_score)
            dist_score = sc
        
    if gpn.remove_useless_edge():
        pos = embedded_viz(gpn)
        quick_export(gpn, pos, root + str(i).zfill(4) + "_d.png")
        print("AAAAAAAAAAAAAA - Remuseless")
        gpn.update_all_dist()
        gpn.print_dist_metrics()
        
    if i % 100 == 0:
        print()
        print("Iteration", i, "ratio", ratio, "density", gpn.density())

In [ ]:
lA, lB, lC = gpn.get_all_dists()
plt.subplot(3, 1, 1)
plt.hist(lA)
plt.subplot(3, 1, 2)
plt.hist(lB)
plt.subplot(3, 1, 3)
plt.hist(lC)

In [ ]:
raise

In [ ]:
for x in [(-1, 117, 1), (117, 118, 0), (118, 47, 0), (-1, 120, 0), (120, 47, 0)]:
    print(gpn.dual(x))

In [ ]:
gpn.G.nodes[9], gpn.G.nodes[8], gpn.G.nodes[0]

In [ ]:
gpn.show_all()

In [ ]:
pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=10)
gpn.update_all_dist()
gpn.show_dist(pos=pos, figsize=(8, 8))

In [ ]:
gpn.dual((125, 15)), gpn.ngb(15), gpn.ngb(7), gpn.dual((125, 7))

In [ ]:
raise

In [ ]:
list_inner = [gpn.G.degree(n) for n in gpn.G.nodes if not gpn.is_border_node(n)]
list_outer = [gpn.G.degree(n) for n in gpn.G.nodes if gpn.is_border_node(n)]

In [ ]:
plt.hist(list_inner)

In [ ]:
plt.hist(list_outer)

In [ ]:
gpn.show_all()

In [ ]:
raise

In [ ]:
# test 1
def test_func():
    gpn = GrowingPlanarNetwork()
    gpn.init_square(8)
    for i in range(500):
        if p_dupl():
            gpn.duplicate_random_node()
        else:
            gpn.remove_random_node()
    dist = np.array(list(nx.betweenness_centrality(gpn.G).values()))
    return (skew(dist), kurtosis(dist), gpn.G, gpn.D)

for i in range(10):
    S, K, G, D = test_func()
    print(G.size(), D.degree(-1))

In [ ]:
G = gpn.G

In [ ]:
pos = nx.spring_layout(G, k=0.1, iterations=50)
pos = nx.planar_layout(gpn.G)
plt.figure(figsize=(12, 12))
nx.draw_networkx(gpn.G, pos=pos)

In [ ]:
pos = nx.planar_layout(Gstart)
plt.figure(figsize=(12, 12))
nx.draw_networkx(Gstart, pos=pos)

In [ ]:
raise

In [ ]:
gpn.show_all()

In [ ]:
ls_ngb = [len(gpn.ngb(x)) for x in gpn.G.nodes]

In [ ]:
plt.hist(ls_ngb)

In [ ]:
raise

In [ ]:
couple_ngb = [(len(gpn.ngb(x)), x) for x in gpn.G.nodes if len(gpn.ngb(x)) > 4]
couple_ngb

In [ ]:
gpn.stabilize_ngb(10)

In [ ]:
couple_ngb = [(len(gpn.ngb(x)), x) for x in gpn.G.nodes if len(gpn.ngb(x)) > 4]
couple_ngb